# Sentiment Analysis Recommendation System 

## Group: U_Data

    Members:
    
    Aroob Almutairi 
    
    Abdulrahman Alosaimi
    
    Duaa Al saad 
    
    Fatimah Alateeq
    
    Ghaliah mohammedhussein

## Required installation 

In [ ]:
# make sure to create a new environment in anaconda navigator with python version 3.6
# ! pip install pandas
# ! pip install numpy
# ! pip install scikit-learn
# ! pip install matplotlib
# ! pip install seaborn
# ! pip install pyarabic
# ! pip install emoji
# ! pip install pystemmer
# ! pip install optuna==2.3.0
# ! pip install transformers==4.2.1
# ! pip install torch

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

/Users/Ghaliah_Maher/opt/anaconda3/envs/DL/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No GPU available, using the CPU instead.


## Import needed packages

In [ ]:
import numpy as np
import pandas as pd
import pyarabic.araby as ar
import re , emoji, Stemmer, functools, operator, string
import torch , optuna, gc, random, os
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
from transformers import AutoModelForSequenceClassification
from transformers import pipeline

import logging
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline 


## Load the Data

In [ ]:
# df_NLP_new= pd.read_csv('Cleaned_data_set/clean_NLP_data_set.csv')
# df_NLP_new.drop('Unnamed: 0',axis=1,inplace=True)
# df_NLP_new

,tweet,label
0,اف مافي شي راضي يزبط,anger
1,مبسوطه واحساس بالراحه وتنفس مريح,joy
2,متعاطفه مرا مع هذا الشخص,sympathy
3,أنا مكتئبة جدا احتاج عزلة عن الناس,sadness
4,احساس بعدم الراحه قلق وشك نبضات قلب سريعه تنفس...,fear
...,...,...
4016,انا اقوى من الملل,bored
4017,مره يعصب صدق,anger
4018,اشعر بنيران تخرج من عيناي,anger
4019,مصدومة,surprise


## Split data

In [ ]:
# train_data=df_NLP_new.iloc[0:3500]
# test_data=df_NLP_new.iloc[3501:4020]

In [ ]:
# train_data['label'].unique()

array(['anger', 'joy', 'sympathy', 'sadness', 'fear', 'bored', 'surprise',
       'none', 'love'], dtype=object)

In [ ]:
# df_NLP_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4021 entries, 0 to 4020
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   4021 non-null   object
 1   label   4021 non-null   object
dtypes: object(2)
memory usage: 63.0+ KB


## Prepare Training Set

In [ ]:
# Tweets_Text_Col_Train = "tweet"
# Tweets_Sentiment_Col_Train = "label"
# Tweets_Ids_Col_Test = "tweet"
# Tweets_Text_Col_Test = "label"

In [ ]:
# Extra_Len = 6 # an extra padding in length , found to be useful for increasing F-score
# Max_Len =int( train_data[Tweets_Text_Col_Train].str.split().str.len().max() + Extra_Len)
# print(Max_Len)

# Test_Size = 0
# Rand_Seed = 42 

# train_set, evaluation_set = train_test_split( train_data, test_size= 0.25, random_state= Rand_Seed)

# print("Train set: ")
# print(train_set[Tweets_Sentiment_Col_Train].value_counts())
# print("---------------------------")
# print ("Test set: ")
# print (evaluation_set[Tweets_Sentiment_Col_Train].value_counts())



19
Train set: 
bored       782
joy         281
anger       276
surprise    266
fear        258
love        255
sympathy    247
sadness     245
none         15
Name: label, dtype: int64
---------------------------
Test set: 
bored       268
sadness     100
anger        95
love         91
sympathy     90
surprise     76
fear         74
joy          73
none          8
Name: label, dtype: int64


## Preparing BERTModel Classes

In [ ]:
# Model_Used = "UBC-NLP/MARBERT"
# Task_Name = "classification"

# class Dataset:
#     def __init__(
#         self,
#         name,
#         train,
#         test,
#         label_list,
#     ):
#         self.name = name
#         self.train = train
#         self.test = test
#         self.label_list = label_list
        
# class BERTModelDataset(Dataset):
#     def __init__(self, text, target, model_name, max_len, label_map):
#       super(BERTModelDataset).__init__()
#       self.text = text
#       self.target = target
#       self.tokenizer_name = model_name
#       self.tokenizer = AutoTokenizer.from_pretrained(model_name)
#       self.max_len = max_len
#       self.label_map = label_map
  
#     def __len__(self):
#       return len(self.text)

#     def __getitem__(self,item):
#       text = str(self.text[item])
#       text = " ".join(text.split())
    
#       encoded_review = self.tokenizer.encode_plus(
#       text,
#       max_length= self.max_len,
#       add_special_tokens= True,
#       return_token_type_ids=False,
#       pad_to_max_length=True,
#       truncation='longest_first',
#       return_attention_mask=True,
#       return_tensors='pt'
#     )
#       input_ids = encoded_review['input_ids'].to(device)
#       attention_mask = encoded_review['attention_mask'].to(device)

#       return InputFeatures(input_ids=input_ids.flatten(), attention_mask=attention_mask.flatten(), label=self.label_map[self.target[item]])

## Defining Needed Methods for Training and Testing

In [ ]:
# def model_init():
#   return AutoModelForSequenceClassification.from_pretrained(Model_Used, return_dict=True, num_labels=len(label_map))

# def compute_metrics(p): #p should be of type EvalPrediction
#   preds = np.argmax(p.predictions, axis=1)
#   assert len(preds) == len(p.label_ids)
#   print(classification_report(p.label_ids,preds))
#   #print(confusion_matrix(p.label_ids,preds))

#   macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[1,2])
#   macro_f1 = f1_score(p.label_ids,preds,average='macro')
#   macro_precision = precision_score(p.label_ids,preds,average='macro')
#   macro_recall = recall_score(p.label_ids,preds,average='macro')
#   acc = accuracy_score(p.label_ids,preds)
#   return {
#       'macro_f1' : macro_f1,
#       'macro_f1_pos_neg' : macro_f1_pos_neg,  
#       'macro_precision': macro_precision,
#       'macro_recall': macro_recall,
#       'accuracy': acc
#   }

# def set_seed(seed):
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False
#     np.random.seed(seed)
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)


## Build Train and Test Data Sets

In [ ]:
# label_list = list(train_set[Tweets_Sentiment_Col_Train].unique())

# print(label_list)
# print(train_set[Tweets_Sentiment_Col_Train].value_counts())

# data_set = Dataset( "KAUST", train_set, evaluation_set, label_list )

# label_map = { v:index for index, v in enumerate(label_list) }
# print(label_map)

# train_dataset = BERTModelDataset(train_set[Tweets_Text_Col_Train].to_list(),
#                                  train_set[Tweets_Sentiment_Col_Train].to_list(),Model_Used,Max_Len,label_map)

# evaluation_dataset = BERTModelDataset(evaluation_set[Tweets_Text_Col_Train].to_list(),
#                                       evaluation_set[Tweets_Sentiment_Col_Train].to_list(),Model_Used,Max_Len,label_map)



['anger', 'bored', 'surprise', 'sadness', 'fear', 'sympathy', 'joy', 'love', 'none']
bored       782
joy         281
anger       276
surprise    266
fear        258
love        255
sympathy    247
sadness     245
none         15
Name: label, dtype: int64
{'anger': 0, 'bored': 1, 'surprise': 2, 'sadness': 3, 'fear': 4, 'sympathy': 5, 'joy': 6, 'love': 7, 'none': 8}


## Definge Training Arguments

In [ ]:
# #define training arguments
# training_args = TrainingArguments("./train")
# training_args.lr_scheduler_type = 'cosine'
# training_args.evaluate_during_training = True
# training_args.adam_epsilon =1e-8 
# # if Use_Train_Extended_Data :
# #     training_args.learning_rate = 1.215e-05 # use this with extended data
# # else:
# #     training_args.learning_rate = 1.78255000000000001e-05 # use this with org data  
# training_args.fp16 = True
# training_args.per_device_train_batch_size = 16 #64 
# training_args.per_device_eval_batch_size = 16 # 64 
# training_args.gradient_accumulation_steps = 2
# training_args.num_train_epochs= 2
# training_args.warmup_steps = 0 
# training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
# training_args.save_steps = 100000 
# training_args.seed = 42 
# training_args.disable_tqdm = False

## Build Trainer

In [ ]:
# training_args.dataloader_pin_memory = False
# gc.collect()
# torch.cuda.empty_cache()
# set_seed(Rand_Seed) 

# trainer = Trainer(
#     model = model_init(),
#     args = training_args,
#     train_dataset = train_dataset,
#     eval_dataset= evaluation_dataset,
#     compute_metrics=compute_metrics
# )

# print(training_args.seed)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

42


/Users/Ghaliah_Maher/opt/anaconda3/envs/DL/lib/python3.6/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


## Train 

In [ ]:
# print(Max_Len)
# print(training_args.learning_rate)
# print(training_args.adam_epsilon)
# print(training_args.warmup_steps)
# #wandbkey if needed (depend on the transformers package version) = 0a58b374c46a154de1ba77c8634c6be279a9dcdb
# trainer.train()

/Users/Ghaliah_Maher/opt/anaconda3/envs/DL/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/Users/Ghaliah_Maher/opt/anaconda3/envs/DL/lib/python3.6/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


19
5e-05
1e-08
0


Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
0,No log,0.159235,0.861664,0.961217,0.859766,0.865301,0.969143,70.964100,12.330000
1,No log,0.104007,0.871661,0.987886,0.869456,0.874100,0.977143,70.228000,12.459000


              precision    recall  f1-score   support

           0       0.98      0.96      0.97        95
           1       0.98      1.00      0.99       268
           2       0.88      0.99      0.93        76
           3       0.94      0.99      0.97       100
           4       0.99      1.00      0.99        74
           5       0.99      0.87      0.92        90
           6       1.00      0.99      0.99        73
           7       0.98      1.00      0.99        91
           8       0.00      0.00      0.00         8

    accuracy                           0.97       875
   macro avg       0.86      0.87      0.86       875
weighted avg       0.96      0.97      0.96       875



/Users/Ghaliah_Maher/opt/anaconda3/envs/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Ghaliah_Maher/opt/anaconda3/envs/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Ghaliah_Maher/opt/anaconda3/envs/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        95
           1       0.98      1.00      0.99       268
           2       0.99      0.99      0.99        76
           3       0.95      0.99      0.97       100
           4       0.99      1.00      0.99        74
           5       0.95      0.91      0.93        90
           6       1.00      1.00      1.00        73
           7       0.99      1.00      0.99        91
           8       0.00      0.00      0.00         8

    accuracy                           0.98       875
   macro avg       0.87      0.87      0.87       875
weighted avg       0.97      0.98      0.97       875



/Users/Ghaliah_Maher/opt/anaconda3/envs/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Ghaliah_Maher/opt/anaconda3/envs/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Ghaliah_Maher/opt/anaconda3/envs/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

TrainOutput(global_step=164, training_loss=0.4587315815251048, metrics={'train_runtime': 1447.62, 'train_samples_per_second': 0.113, 'total_flos': 97446121900290, 'epoch': 1.99})

## Saving the Model

In [ ]:
# trainer.save_model('final_model/model')
# trainer.model.save_pretrained('final_model/model_final')
# train_dataset.tokenizer.save_pretrained('final_model/model_final')

('final_model/model_final/tokenizer_config.json',
 'final_model/model_final/special_tokens_map.json',
 'final_model/model_final/vocab.txt',
 'final_model/model_final/added_tokens.json')

## Load the Model

In [ ]:
load_model = AutoModelForSequenceClassification.from_pretrained("final_model/model_final")
 
load_tokenizer = AutoTokenizer.from_pretrained("final_model/model_final")

## Testing the Model 

In [ ]:
my_pipeline  = pipeline("sentiment-analysis", model=load_model, tokenizer=load_tokenizer)# outputs a list of dicts 
result = my_pipeline("غاضب")

In [ ]:
sentiment = {'LABEL_0': "anger",
             'LABEL_1': 'bored',
             'LABEL_2': 'surprise',
             'LABEL_3': 'sadness',
             'LABEL_4': 'fear',
             'LABEL_5' :'sympathy',
             'LABEL_6' :'joy',
             'LABEL_7': 'love',
             'LABEL_8': 'none'}

# {'anger': 0, 'bored': 1, 'surprise': 2, 'sadness': 3, 'fear': 4, 'sympathy': 5, 'joy': 6, 'love': 7, 'none': 8}


In [ ]:
x=result[0]['label']
print(x)
for key,value in sentiment.items():
  if x == key:
    x=value
print(x)

LABEL_0
anger


## Recommendation System

In [ ]:
dff=pd.read_csv('Cleaned_data_set/clean_stc_data_set.csv')

In [ ]:
emotion = dict(
    {
        'anger': ['Action','Wrestling','Crime','Thriller'] ,
        'love': ['Romance','Drama','Family'],
        'joy': ['Animation','Comedy','Adventure'],
        'surprise': ['Animation','Thriller','Horror'] ,
        'sadness': ['Drama','Comedy','Family','Animation'],
        'fear': ['Comedy','Family','Animation'],
        'sympathy': ['Drama','Documentary','Biography','Family'] ,
        'bored': ['Animation','Adventure','Thriller','Crime','Sci-Fi'],
        'none': ['Documentary']
    })

In [ ]:
def recommend(x):
    dic = {}
    for i,n in emotion.items():
        if x == i:
            for j in n:
                v=dff.loc[dff['program_genre']==j][['original_name','normalized_score']].drop_duplicates(subset = "original_name").sort_values(by='normalized_score',ascending=False)
                v=v.head(3)
                for a in v.iterrows():
                    dic[j,a[1][0]]= a[1][1]  

    for key,value in dic.items():
        print(key,value)

In [ ]:
x = 'anger'

In [ ]:
recommend(x)

('Action', 'The Amazing pider-Man') 10
('Action', 'Teen Wolf') 10
('Action', 'The Expendables') 8
('Wrestling', 'WWE Royal Rumble') 10
('Crime', 'Inside Man') 10
('Crime', 'Criminal Minds') 10
('Crime', 'Scarface') 5
('Thriller', 'Drive Angry') 10
('Thriller', 'Dexter') 10
('Thriller', 'Red Dragon') 9
